<a href="https://colab.research.google.com/github/ikanx101/AppSheet-Sales/blob/main/AppSheet_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# _AppSheet Converter_ 2024

_Converter_ ini dibuat untuk mengubah data hasil _input_ __AppSheet__ ke bentuk tabular yang lebih _excel-user-friendly_.

Cara pakai:

1. _Upload_ dua _files_ yang dibutuhkan, yakni:
    - Data __AppSheet__.
    - Data _pricelist_.
1. Isi _form_ sesuai dengan nama _files_ tersebut.
1. Klik `Runtime` >> `Run all`.
1. Tunggu hingga proses selesai (sampai muncul pesan `PROSES SELESAI`). Kira-kira akan memakan waktu 5-10 menit.
1. Akan muncul dua _files_ baru bernama `Omzet_converted.xlsx` dan `AV_converted.xlsx`. Silakan di-_download_.

---

Catatan: Karena yang kita gunakan saat ini adalah _server_ gratisnya Google, jadi kita harus selalu _install_ beberapa _packages_ setiap kali _run_ skrip ini. Konsekuensinya _runtime_-nya lebih lama.

_Last Update_: 27 Desember 2023

In [ ]:
#@title Masukkan Nama Files

rm(list=ls())
nama_file_appsheet <- "Lalala" #@param {type:"string"}
nama_file_pricelist <- "Template Harga 2.xlsx" #@param {type:"string"}

install.packages("reshape2")
install.packages("janitor")
install.packages("openxlsx")

In [ ]:
# @title
# ==============================================================================
# libraries yang terlibat
library(dplyr)
library(tidyr)
library(parallel)
library(readxl)

# menentukan berapa banyak cores yang terlibat
n_core = detectCores()

# si warna merah yang tak diperlukan AV
# warna_merah = readLines("warna merah.txt") %>% as.numeric()

# si warna merah yang tak diperlukan availability
# warna_merah_av = readLines("warna merah - av.txt") %>% as.numeric()

# function ntuk mengembalikan nama produk
benerin = function(tes){
  gsub("_"," ",tes) %>% toupper()
  }

# function untuk benerin nama kolom
nama_judul = function(tes){
  benerin(tes) %>% stringr::str_to_title()
  }
# ==============================================================================


# ==============================================================================
# baca file harga
file_harga = "Template Harga 2.xlsx"
df_harga   =
  read_excel(file_harga) %>%
  janitor::clean_names() %>%
  rename(item_penjualan = nama_item)

# baca file yang hendak dikonversi
file_conv  = "Call (7).xlsx"
df_raw     =
  read_excel(file_conv) %>%
  janitor::clean_names()

# ambil nama kolom untuk omset
nama_kolom = colnames(df_raw)
# pertengahan pg3 dan pg5 harus kita hapus
awal  = which(nama_kolom == "pg3")
akhir = which(nama_kolom == "pg_5")
hapus = awal:akhir

# ini yang perlu diambil
df_raw = df_raw[-hapus]
# ==============================================================================


# ==============================================================================
# tahap 1
# kita pisahkan untuk df omset terlebih dahulu
selection = c("waktu","tanggal","bulan","nama_mds","id_mds","area_mds",
              "region_mds","pic","kode_customer","nama_customer",
              "no_hp_customer","kecamatan","kabupaten","provinsi",
              "detail_klasifikasi","klasifikasi","sekolah",
              "koordinat_ro","koordinat_call","jarak_meter",
              "kesesuaian_titik","peserta_display_wow_operator",
              "peserta_loyalty_sachet","project_1","project_2",
              "transaksi_penjualan","av_item","check_out","durasi")

# pemisahan pertama
df_omset_raw_1 =
  df_raw %>%
  select(id,contains(selection))
# colnames(df_omset_raw_1)

# pemisahan kedua
# ambil nama kolom untuk omset
nama_kolom = colnames(df_raw)
# pertengahan pg3 dan pg5 harus kita hapus
awal   = which(nama_kolom == "pg")
akhir  = which(nama_kolom == "pg_8")
simpan = c(1,awal:akhir)

df_omset_raw_2 =
  df_raw[simpan] %>%
  select(-contains("pg")) %>%
  reshape2::melt(id.vars = "id") %>%
  filter(!is.na(value)) %>%
  rename(item_penjualan = variable,
         qty_penjualan  = value) %>%
  rowwise() %>%
  mutate(item_penjualan = benerin(item_penjualan)) %>%
  ungroup() %>%
  merge(df_harga,by = "item_penjualan") %>%
  mutate(omzet = harga * qty_penjualan)

# kita gabung kembali ke format yang diinginkan
df_gabung =
  merge(df_omset_raw_1,df_omset_raw_2,by = "id") %>%
  relocate(brand,sub_brand,harga,.after = "item_penjualan") %>%
  relocate(av_item,check_out,durasi,.after = "omzet") %>%
  # ini yang kita hapus
  select(-check_out,-durasi)

# benerin nama kolom finalnya
colnames(df_gabung) = nama_judul(colnames(df_gabung))

openxlsx::write.xlsx(df_gabung,file = "Omzet_converted.xlsx")
# ==============================================================================


# ==============================================================================
# tahap 2
# baca file yang hendak dikonversi
df_raw     =
  read_excel(file_conv) %>%
  janitor::clean_names()

# ambil nama kolom untuk av
nama_kolom = colnames(df_raw)
# pertengahan pg3 dan pg5 harus kita hapus
awal     = which(nama_kolom == "pg3")
akhir    = which(nama_kolom == "pg_5")
ambil_av = c(1,awal:akhir)

# kita lakukan pemecahan kembali
# pemecahan 1
# pemisahan pertama
df_av_raw_1 =
  df_raw %>%
  select(id,contains(selection))

# pemisahan kedua
df_av_raw_2 =
  df_raw[ambil_av] %>%
  reshape2::melt(id.vars = "id") %>%
  mutate(value = as.numeric(value)) %>%
  filter(value > 0) %>%
  select(-value) %>%
  rename(availability_item = variable) %>%
  mutate(availability_item = benerin(availability_item))

# kita gabung kembali ke format yang diinginkan
df_gabung =
  merge(df_av_raw_1,df_av_raw_2,by = "id") %>%
  relocate(availability_item,.before = "peserta_display_wow_operator") %>%
  relocate(check_out,durasi,.after = "project_2") %>%
  # ini yang kita hapus
  select(-check_out,-durasi)

# benerin nama kolom finalnya
colnames(df_gabung) = nama_judul(colnames(df_gabung))

openxlsx::write.xlsx(df_gabung,file = "AV_converted.xlsx")
# ==============================================================================

In [ ]:
#@title Status
print("PROSES SELESAI")
print("file hasil konversi tersimpan dalam nama: Omzet_converted.xlsx dan AV_converted.xlsx")